In [1]:
'''
This file plots CMIP5 RCP - HIST over South Australia
/Users/earl/Desktop/Yang
and places the outputs in
/Users/earl/Dropbox/CMIP5/figures

Earl Duran 
created: 19-Mar-18
e.duran@unsw.edu.au
'''

import cosima_cookbook as cc
import os
import xarray as xr
import numpy as np
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
import matplotlib
import sys
import pickle
import itertools
def find_nearest_index(array, value):
    return int((np.abs(array - value)).argmin())
from scipy import interpolate
from scipy import stats
from dask.distributed import Client
import pandas as pd
from glob import glob as gb
import gsw

import cartopy.crs as ccrs
import cartopy.feature as cft
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
def arange(start,end,stride=1):
    return np.arange(start,end+0.00000001,stride)

import sys, os
sys.path.append(os.path.join(os.getcwd(), '..'))  # so we can import ../exptdata
import exptdata
print('Available exptdata keys: ', [k for k in exptdata.exptdict.keys()])

session = cc.database.create_session()

Available exptdata keys:  ['1deg', '025deg', '01deg']


In [2]:
%%javascript
IPython.notebook.kernel.execute('nb_name = ' + '"' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [3]:
client = Client()
client

Client Scheduler: tcp://127.0.0.1:44923 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 8 Memory: 33.67 GB


In [4]:
# input_path = '/g/data/e14/erd561/NOAA_OI_SST_V2/'

output_path = '/g/data/e14/erd561/Australia_3/'

In [5]:
%%time
lonW = 145-360
lonE = 175-360
latL = -34
ekey='1deg'
expt = exptdata.exptdict[ekey]['expt']
print(expt)

temp_ym34_1_mod_month = cc.querying.getvar(expt=expt,
                                     session=session,
                                     ncfile='ocean.nc',
                                     variable='temp').\
interp(yt_ocean=latL).\
sel(xt_ocean=slice(lonW,lonE)).\
sel(st_ocean=slice(0,2100))-273.15
print(temp_ym34_1_mod_month)

temp_ym34_1_mod_month.load()
print(temp_ym34_1_mod_month)

xt_ocean = temp_ym34_1_mod_month.xt_ocean
xt_ocean_corrected = xt_ocean + 360
temp_ym34_1_mod_month = temp_ym34_1_mod_month.assign_coords(xt_ocean=xt_ocean_corrected)
st_ocean = temp_ym34_1_mod_month.st_ocean
st_ocean_corrected = -st_ocean
temp_ym34_1_mod_month = temp_ym34_1_mod_month.assign_coords(st_ocean=st_ocean_corrected)
print(temp_ym34_1_mod_month)

temp_ym34_1_mod = temp_ym34_1_mod_month.groupby('time.year').mean('time')
print(temp_ym34_1_mod)

temp_ym34_1_mod = temp_ym34_1_mod.sel(year=slice(2004+60*4,2017+60*4))
print(temp_ym34_1_mod)

1deg_jra55v13_iaf_spinup1_B1


/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/cosima_cookbook/querying.py:96: FutureWarning: In xarray version 0.14 the default behaviour of `open_mfdataset`
will change. To retain the existing behavior, pass
combine='nested'. To use future default behavior, pass
combine='by_coords'. See
http://xarray.pydata.org/en/stable/combining.html#combining-multi

  preprocess=lambda d: d[variable].to_dataset() if variable not in d.coords else d)
/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/xarray/backends/api.py:934: FutureWarning: The datasets supplied have global dimension coordinates. You may want
to use the new `combine_by_coords` function (or the
`combine='by_coords'` option to `open_mfdataset`) to order the datasets
before concatenation. Alternatively, to continue concatenating based
on the order the datasets are supplied in future, please use the new
`combine_nested` function (or the `combine='nested'` option t

<xarray.DataArray 'temp' (time: 300, st_ocean: 35, xt_ocean: 30)>
dask.array<sub, shape=(300, 35, 30), dtype=float32, chunksize=(1, 25, 30), chunktype=numpy.ndarray>
Coordinates:
  * st_ocean  (st_ocean) float64 1.152 3.649 6.565 ... 1.854e+03 2.046e+03
  * xt_ocean  (xt_ocean) float64 -214.5 -213.5 -212.5 ... -187.5 -186.5 -185.5
  * time      (time) object 1958-07-02 12:00:00 ... 2257-07-02 12:00:00
    yt_ocean  int64 -34
<xarray.DataArray 'temp' (time: 300, st_ocean: 35, xt_ocean: 30)>
array([[[        nan,         nan,         nan, ..., 18.38574048,
         18.70731909, 18.8863487 ],
        [        nan,         nan,         nan, ..., 18.37032008,
         18.68126044, 18.8653666 ],
        [        nan,         nan,         nan, ..., 18.35224667,
         18.64779691, 18.82870922],
        ...,
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 n

/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)


In [6]:
year = temp_ym34_1_mod.year
depth_1_mod = temp_ym34_1_mod.st_ocean
lon_1_mod = temp_ym34_1_mod.xt_ocean

temp_ym34_1_trans_mod = temp_ym34_1_mod
print(np.shape(temp_ym34_1_trans_mod))
temp_ym34_1_slope_mod = xr.DataArray(
    np.zeros([np.shape(depth_1_mod)[0], np.shape(lon_1_mod)[0]]), dims=(
    'depth_1', 'lon_1'), coords=[depth_1_mod, lon_1_mod], name='temp_ym34_1')
temp_ym34_1_p_value_mod = xr.DataArray(
    np.zeros([np.shape(depth_1_mod)[0], np.shape(lon_1_mod)[0]]), dims=(
    'depth_1', 'lon_1'), coords=[depth_1_mod, lon_1_mod], name='temp_ym34_1')
temp_ym34_1_std_err_mod = xr.DataArray(
    np.zeros([np.shape(depth_1_mod)[0], np.shape(lon_1_mod)[0]]), dims=(
    'depth_1', 'lon_1'), coords=[depth_1_mod, lon_1_mod], name='temp_ym34_1')
for iid, i in enumerate(depth_1_mod):
    for jid, j in enumerate(lon_1_mod):
        temp_ym34_1_slope_mod[iid, jid], _, _, temp_ym34_1_p_value_mod[iid, jid], temp_ym34_1_std_err_mod[iid, jid] = \
        stats.linregress(year, temp_ym34_1_trans_mod[:, iid, jid])
    print('depth_1_mod ' + str(np.array(i)))
print(temp_ym34_1_slope_mod)

(14, 35, 30)
depth_1_mod -1.15174989182586
depth_1_mod -3.6486741197120196
depth_1_mod -6.564918471676153
depth_1_mod -9.970869194872916
depth_1_mod -13.948708373749158
depth_1_mod -18.594379460593196
depth_1_mod -24.019873617545198
depth_1_mod -30.355885242559452
depth_1_mod -37.754889932903495
depth_1_mod -46.39470194845016
depth_1_mod -56.48256969937929
depth_1_mod -68.25986482598941
depth_1_mod -82.00740970018322
depth_1_mod -98.05146439730592
depth_1_mod -116.77034928747688
depth_1_mod -138.60160118643546
depth_1_mod -164.04943216631892
depth_1_mod -193.69205664349184
depth_1_mod -228.1881444592732
depth_1_mod -268.2812141854562
depth_1_mod -314.800181990729
depth_1_mod -368.65354740782504
depth_1_mod -430.8139438640983
depth_1_mod -502.28929308570173
depth_1_mod -584.0771003035616
depth_1_mod -677.1002526914099
depth_1_mod -782.1267221440287
depth_1_mod -899.6817765561261
depth_1_mod -1029.9681126743835
depth_1_mod -1172.813371535859
depth_1_mod -1327.6620295174746
depth_1_mod -1

In [7]:
%%time
latL = -34
ekey='025deg'
expt = exptdata.exptdict[ekey]['expt']
print(expt)

temp_ym34_025_mod_month = cc.querying.getvar(expt=expt,
                                     session=session,
                                     ncfile='ocean.nc',
                                     variable='temp').\
interp(yt_ocean=latL).\
sel(xt_ocean=slice(lonW,lonE)).\
sel(st_ocean=slice(0,2100))-273.15
print(temp_ym34_025_mod_month)

temp_ym34_025_mod_month.load()
print(temp_ym34_025_mod_month)

xt_ocean = temp_ym34_025_mod_month.xt_ocean
xt_ocean_corrected = xt_ocean + 360
temp_ym34_025_mod_month = temp_ym34_025_mod_month.assign_coords(xt_ocean=xt_ocean_corrected)
st_ocean = temp_ym34_025_mod_month.st_ocean
st_ocean_corrected = -st_ocean
temp_ym34_025_mod_month = temp_ym34_025_mod_month.assign_coords(st_ocean=st_ocean_corrected)
print(temp_ym34_025_mod_month)

temp_ym34_025_mod = temp_ym34_025_mod_month.groupby('time.year').mean('time')
print(temp_ym34_025_mod)

temp_ym34_025_mod = temp_ym34_025_mod.sel(year=slice(2004+60*4,2017+60*4))
print(temp_ym34_025_mod)

025deg_jra55v13_iaf_gmredi6


/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/cosima_cookbook/querying.py:96: FutureWarning: In xarray version 0.14 the default behaviour of `open_mfdataset`
will change. To retain the existing behavior, pass
combine='nested'. To use future default behavior, pass
combine='by_coords'. See
http://xarray.pydata.org/en/stable/combining.html#combining-multi

  preprocess=lambda d: d[variable].to_dataset() if variable not in d.coords else d)
/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/xarray/backends/api.py:934: FutureWarning: The datasets supplied have global dimension coordinates. You may want
to use the new `combine_by_coords` function (or the
`combine='by_coords'` option to `open_mfdataset`) to order the datasets
before concatenation. Alternatively, to continue concatenating based
on the order the datasets are supplied in future, please use the new
`combine_nested` function (or the `combine='nested'` option t

<xarray.DataArray 'temp' (time: 300, st_ocean: 35, xt_ocean: 120)>
dask.array<sub, shape=(300, 35, 120), dtype=float32, chunksize=(1, 10, 92), chunktype=numpy.ndarray>
Coordinates:
  * st_ocean  (st_ocean) float64 1.152 3.649 6.565 ... 1.854e+03 2.046e+03
  * xt_ocean  (xt_ocean) float64 -214.9 -214.6 -214.4 ... -185.6 -185.4 -185.1
  * time      (time) object 1958-06-30 12:00:00 ... 2257-06-30 12:00:00
    yt_ocean  int64 -34
<xarray.DataArray 'temp' (time: 300, st_ocean: 35, xt_ocean: 120)>
array([[[        nan,         nan,         nan, ..., 19.46310667,
         19.46670969, 19.46298203],
        [        nan,         nan,         nan, ..., 19.45372717,
         19.45691914, 19.45284397],
        [        nan,         nan,         nan, ..., 19.43777765,
         19.4405542 , 19.43636756],
        ...,
        [        nan,         nan,         nan, ...,  2.79938031,
          2.81779342,  2.85034273],
        [        nan,         nan,         nan, ...,         nan,
               

/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)


In [8]:
year = temp_ym34_025_mod.year
depth_025_mod = temp_ym34_025_mod.st_ocean
lon_025_mod = temp_ym34_025_mod.xt_ocean

temp_ym34_025_trans_mod = temp_ym34_025_mod
print(np.shape(temp_ym34_025_trans_mod))
temp_ym34_025_slope_mod = xr.DataArray(
    np.zeros([np.shape(depth_025_mod)[0], np.shape(lon_025_mod)[0]]), dims=(
    'depth_025', 'lon_025'), coords=[depth_025_mod, lon_025_mod], name='temp_ym34_025')
temp_ym34_025_p_value_mod = xr.DataArray(
    np.zeros([np.shape(depth_025_mod)[0], np.shape(lon_025_mod)[0]]), dims=(
    'depth_025', 'lon_025'), coords=[depth_025_mod, lon_025_mod], name='temp_ym34_025')
temp_ym34_025_std_err_mod = xr.DataArray(
    np.zeros([np.shape(depth_025_mod)[0], np.shape(lon_025_mod)[0]]), dims=(
    'depth_025', 'lon_025'), coords=[depth_025_mod, lon_025_mod], name='temp_ym34_025')
for iid, i in enumerate(depth_025_mod):
    for jid, j in enumerate(lon_025_mod):
        temp_ym34_025_slope_mod[iid, jid], _, _, temp_ym34_025_p_value_mod[iid, jid], temp_ym34_025_std_err_mod[iid, jid] = \
        stats.linregress(year, temp_ym34_025_trans_mod[:, iid, jid])
    print('depth_025_mod ' + str(np.array(i)))
print(temp_ym34_025_slope_mod)

(14, 35, 120)
depth_025_mod -1.15174989182586
depth_025_mod -3.6486741197120196
depth_025_mod -6.564918471676153
depth_025_mod -9.970869194872916
depth_025_mod -13.948708373749158
depth_025_mod -18.594379460593196
depth_025_mod -24.019873617545198
depth_025_mod -30.355885242559452
depth_025_mod -37.754889932903495
depth_025_mod -46.39470194845016
depth_025_mod -56.48256969937929
depth_025_mod -68.25986482598941
depth_025_mod -82.00740970018322
depth_025_mod -98.05146439730592
depth_025_mod -116.77034928747688
depth_025_mod -138.60160118643546
depth_025_mod -164.04943216631892
depth_025_mod -193.69205664349184
depth_025_mod -228.1881444592732
depth_025_mod -268.2812141854562
depth_025_mod -314.800181990729
depth_025_mod -368.65354740782504
depth_025_mod -430.8139438640983
depth_025_mod -502.28929308570173
depth_025_mod -584.0771003035616
depth_025_mod -677.1002526914099
depth_025_mod -782.1267221440287
depth_025_mod -899.6817765561261
depth_025_mod -1029.9681126743835
depth_025_mod -117

In [9]:
%%time
latL = -34
ekey='01deg'
expt = exptdata.exptdict[ekey]['expt']
print(expt)

temp_ym34_01_mod_month = cc.querying.getvar(expt=expt,
                                     session=session,
                                     ncfile='ocean.nc',
                                     variable='temp').\
sel(yt_ocean=latL, method='nearest').\
sel(xt_ocean=slice(lonW,lonE)).\
sel(st_ocean=slice(0,2100))-273.15
print(temp_ym34_01_mod_month)

temp_ym34_01_mod_month.load()
print(temp_ym34_01_mod_month)

xt_ocean = temp_ym34_01_mod_month.xt_ocean
xt_ocean_corrected = xt_ocean + 360
temp_ym34_01_mod_month = temp_ym34_01_mod_month.assign_coords(xt_ocean=xt_ocean_corrected)
st_ocean = temp_ym34_01_mod_month.st_ocean
st_ocean_corrected = -st_ocean
temp_ym34_01_mod_month = temp_ym34_01_mod_month.assign_coords(st_ocean=st_ocean_corrected)
print(temp_ym34_01_mod_month)

temp_ym34_01_mod = temp_ym34_01_mod_month.groupby('time.year').mean('time')
print(temp_ym34_01_mod)

temp_ym34_01_mod = temp_ym34_01_mod.sel(year=slice(2004,2017))
print(temp_ym34_01_mod)

01deg_jra55v13_iaf


/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/cosima_cookbook/querying.py:96: FutureWarning: In xarray version 0.14 the default behaviour of `open_mfdataset`
will change. To retain the existing behavior, pass
combine='nested'. To use future default behavior, pass
combine='by_coords'. See
http://xarray.pydata.org/en/stable/combining.html#combining-multi

  preprocess=lambda d: d[variable].to_dataset() if variable not in d.coords else d)
/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/xarray/backends/api.py:934: FutureWarning: The datasets supplied have global dimension coordinates. You may want
to use the new `combine_by_coords` function (or the
`combine='by_coords'` option to `open_mfdataset`) to order the datasets
before concatenation. Alternatively, to continue concatenating based
on the order the datasets are supplied in future, please use the new
`combine_nested` function (or the `combine='nested'` option t

<xarray.DataArray 'temp' (time: 396, st_ocean: 55, xt_ocean: 300)>
dask.array<sub, shape=(396, 55, 300), dtype=float32, chunksize=(1, 7, 150), chunktype=numpy.ndarray>
Coordinates:
    yt_ocean  float64 -33.97
  * st_ocean  (st_ocean) float64 0.5413 1.681 2.94 ... 1.858e+03 2.007e+03
  * xt_ocean  (xt_ocean) float64 -214.9 -214.8 -214.7 ... -185.2 -185.1 -185.0
  * time      (time) object 1985-01-14 12:00:00 ... 2017-12-14 12:00:00
<xarray.DataArray 'temp' (time: 396, st_ocean: 55, xt_ocean: 300)>
array([[[       nan,        nan,        nan, ..., 22.734222 ,
         22.753845 , 22.75769  ],
        [       nan,        nan,        nan, ..., 22.73288  ,
         22.751465 , 22.754517 ],
        [       nan,        nan,        nan, ..., 22.726074 ,
         22.744324 , 22.74762  ],
        ...,
        [       nan,        nan,        nan, ...,  2.2734375,
          2.299408 ,  2.2711182],
        [       nan,        nan,        nan, ...,  2.1019592,
          2.0933228,  2.093689 ],
    

/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)


In [10]:
year = temp_ym34_01_mod.year
depth_01_mod = temp_ym34_01_mod.st_ocean
lon_01_mod = temp_ym34_01_mod.xt_ocean

temp_ym34_01_trans_mod = temp_ym34_01_mod
print(np.shape(temp_ym34_01_trans_mod))
temp_ym34_01_slope_mod = xr.DataArray(
    np.zeros([np.shape(depth_01_mod)[0], np.shape(lon_01_mod)[0]]), dims=(
    'depth_01', 'lon_01'), coords=[depth_01_mod, lon_01_mod], name='temp_ym34_01')
temp_ym34_01_p_value_mod = xr.DataArray(
    np.zeros([np.shape(depth_01_mod)[0], np.shape(lon_01_mod)[0]]), dims=(
    'depth_01', 'lon_01'), coords=[depth_01_mod, lon_01_mod], name='temp_ym34_01')
temp_ym34_01_std_err_mod = xr.DataArray(
    np.zeros([np.shape(depth_01_mod)[0], np.shape(lon_01_mod)[0]]), dims=(
    'depth_01', 'lon_01'), coords=[depth_01_mod, lon_01_mod], name='temp_ym34_01')
for iid, i in enumerate(depth_01_mod):
    for jid, j in enumerate(lon_01_mod):
        temp_ym34_01_slope_mod[iid, jid], _, _, temp_ym34_01_p_value_mod[iid, jid], temp_ym34_01_std_err_mod[iid, jid] = \
        stats.linregress(year, temp_ym34_01_trans_mod[:, iid, jid])
    print('depth_01_mod ' + str(np.array(i)))
print(temp_ym34_01_slope_mod)

(14, 55, 300)
depth_01_mod -0.5412807653916101
depth_01_mod -1.680734679831433
depth_01_mod -2.939952648914465
depth_01_mod -4.331521485149508
depth_01_mod -5.8693504240540255
depth_01_mod -7.5688099200502155
depth_01_mod -9.446884959648713
depth_01_mod -11.522344392803275
depth_01_mod -13.815927932333222
depth_01_mod -16.350552632935347
depth_01_mod -19.151540835805992
depth_01_mod -22.24687175218868
depth_01_mod -25.667459058670023
depth_01_mod -29.447457087455206
depth_01_mod -33.62459841385048
depth_01_mod -38.24056586677453
depth_01_mod -43.341402211041604
depth_01_mod -48.97796096317406
depth_01_mod -55.20640199365842
depth_01_mod -62.088735720175855
depth_01_mod -69.69341978534455
depth_01_mod -78.09601210263148
depth_01_mod -87.37988400213499
depth_01_mod -97.63699685168024
depth_01_mod -108.96874488209758
depth_01_mod -121.48686590184016
depth_01_mod -135.31441999260392
depth_01_mod -150.5868339523135
depth_01_mod -167.45300594349283
depth_01_mod -186.07646019149857
depth_01_m

In [11]:
temp_ym34_1_mod_xr = xr.DataArray(temp_ym34_1_mod.transpose('st_ocean', 'xt_ocean', 'year'), name='temp_ym34_1_mod',
                        coords=[depth_1_mod, lon_1_mod, year], 
                        dims=['st_ocean', 'xt_ocean', 'year'])
print(temp_ym34_1_mod_xr)
temp_ym34_1_mod_xr.to_netcdf(output_path + 'temp_ym34_1_mod.nc')

temp_ym34_1_slope_mod_xr = xr.DataArray(temp_ym34_1_slope_mod, name='temp_ym34_1_slope_mod',
                        coords=[depth_1_mod, lon_1_mod], 
                        dims=['st_ocean', 'xt_ocean'])
print(temp_ym34_1_slope_mod_xr)
temp_ym34_1_slope_mod_xr.to_netcdf(output_path + 'temp_ym34_1_slope_mod.nc')

temp_ym34_1_p_value_mod_xr = xr.DataArray(temp_ym34_1_p_value_mod, name='temp_ym34_1_p_value_mod',
                        coords=[depth_1_mod, lon_1_mod], 
                        dims=['st_ocean', 'xt_ocean'])
print(temp_ym34_1_p_value_mod_xr)
temp_ym34_1_p_value_mod_xr.to_netcdf(output_path + 'temp_ym34_1_p_value_mod.nc')

temp_ym34_1_std_err_mod_xr = xr.DataArray(temp_ym34_1_std_err_mod, name='temp_ym34_1_std_err_mod',
                        coords=[depth_1_mod, lon_1_mod], 
                        dims=['st_ocean', 'xt_ocean'])
print(temp_ym34_1_std_err_mod_xr)
temp_ym34_1_std_err_mod_xr.to_netcdf(output_path + 'temp_ym34_1_std_err_mod.nc')



temp_ym34_025_mod_xr = xr.DataArray(temp_ym34_025_mod.transpose('st_ocean', 'xt_ocean', 'year'), name='temp_ym34_025_mod',
                        coords=[depth_025_mod, lon_025_mod, year], 
                        dims=['st_ocean', 'xt_ocean', 'year'])
print(temp_ym34_025_mod_xr)
temp_ym34_025_mod_xr.to_netcdf(output_path + 'temp_ym34_025_mod.nc')

temp_ym34_025_slope_mod_xr = xr.DataArray(temp_ym34_025_slope_mod, name='temp_ym34_025_slope_mod',
                        coords=[depth_025_mod, lon_025_mod], 
                        dims=['st_ocean', 'xt_ocean'])
print(temp_ym34_025_slope_mod_xr)
temp_ym34_025_slope_mod_xr.to_netcdf(output_path + 'temp_ym34_025_slope_mod.nc')

temp_ym34_025_p_value_mod_xr = xr.DataArray(temp_ym34_025_p_value_mod, name='temp_ym34_025_p_value_mod',
                        coords=[depth_025_mod, lon_025_mod], 
                        dims=['st_ocean', 'xt_ocean'])
print(temp_ym34_025_p_value_mod_xr)
temp_ym34_025_p_value_mod_xr.to_netcdf(output_path + 'temp_ym34_025_p_value_mod.nc')

temp_ym34_025_std_err_mod_xr = xr.DataArray(temp_ym34_025_std_err_mod, name='temp_ym34_025_std_err_mod',
                        coords=[depth_025_mod, lon_025_mod], 
                        dims=['st_ocean', 'xt_ocean'])
print(temp_ym34_025_std_err_mod_xr)
temp_ym34_025_std_err_mod_xr.to_netcdf(output_path + 'temp_ym34_025_std_err_mod.nc')



temp_ym34_01_mod_xr = xr.DataArray(temp_ym34_01_mod.transpose('st_ocean', 'xt_ocean', 'year'), name='temp_ym34_01_mod',
                        coords=[depth_01_mod, lon_01_mod, year], 
                        dims=['st_ocean', 'xt_ocean', 'year'])
print(temp_ym34_01_mod_xr)
temp_ym34_01_mod_xr.to_netcdf(output_path + 'temp_ym34_01_mod.nc')

temp_ym34_01_slope_mod_xr = xr.DataArray(temp_ym34_01_slope_mod, name='temp_ym34_01_slope_mod',
                        coords=[depth_01_mod, lon_01_mod], 
                        dims=['st_ocean', 'xt_ocean'])
print(temp_ym34_01_slope_mod_xr)
temp_ym34_01_slope_mod_xr.to_netcdf(output_path + 'temp_ym34_01_slope_mod.nc')

temp_ym34_01_p_value_mod_xr = xr.DataArray(temp_ym34_01_p_value_mod, name='temp_ym34_01_p_value_mod',
                        coords=[depth_01_mod, lon_01_mod], 
                        dims=['st_ocean', 'xt_ocean'])
print(temp_ym34_01_p_value_mod_xr)
temp_ym34_01_p_value_mod_xr.to_netcdf(output_path + 'temp_ym34_01_p_value_mod.nc')

temp_ym34_01_std_err_mod_xr = xr.DataArray(temp_ym34_01_std_err_mod, name='temp_ym34_01_std_err_mod',
                        coords=[depth_01_mod, lon_01_mod], 
                        dims=['st_ocean', 'xt_ocean'])
print(temp_ym34_01_std_err_mod_xr)
temp_ym34_01_std_err_mod_xr.to_netcdf(output_path + 'temp_ym34_01_std_err_mod.nc')

<xarray.DataArray 'temp_ym34_1_mod' (st_ocean: 35, xt_ocean: 30, year: 14)>
array([[[        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        ...,
        [17.7810167 , 18.21490999, 18.09337927, ..., 18.57039671,
         19.09770669, 18.61383628],
        [18.15830398, 18.6075129 , 18.43489163, ..., 18.92520371,
         19.42534298, 18.99688343],
        [18.36804115, 18.82117404, 18.66376372, ..., 19.0865674 ,
         19.57419918, 19.18966853]],

       [[        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,      

<xarray.DataArray 'temp_ym34_025_slope_mod' (st_ocean: 35, xt_ocean: 120)>
array([[       nan,        nan,        nan, ..., 0.01766785, 0.01964525,
        0.02081491],
       [       nan,        nan,        nan, ..., 0.01749333, 0.01949769,
        0.02068078],
       [       nan,        nan,        nan, ..., 0.01726794, 0.0193433 ,
        0.02056502],
       ...,
       [       nan,        nan,        nan, ..., 0.00222952, 0.00145843,
        0.00350879],
       [       nan,        nan,        nan, ...,        nan,        nan,
               nan],
       [       nan,        nan,        nan, ...,        nan,        nan,
               nan]])
Coordinates:
  * st_ocean  (st_ocean) float64 -1.152 -3.649 -6.565 ... -1.854e+03 -2.046e+03
  * xt_ocean  (xt_ocean) float64 145.1 145.4 145.6 145.9 ... 174.4 174.6 174.9
<xarray.DataArray 'temp_ym34_025_p_value_mod' (st_ocean: 35, xt_ocean: 120)>
array([[       nan,        nan,        nan, ..., 0.36281844, 0.32797765,
        0.30966695],
     

In [12]:
%%time
latL = -42
ekey='1deg'
expt = exptdata.exptdict[ekey]['expt']
print(expt)

temp_ym42_1_mod_month = cc.querying.getvar(expt=expt,
                                     session=session,
                                     ncfile='ocean.nc',
                                     variable='temp').\
interp(yt_ocean=latL).\
sel(xt_ocean=slice(lonW,lonE)).\
sel(st_ocean=slice(0,2100))-273.15
print(temp_ym42_1_mod_month)

temp_ym42_1_mod_month.load()
print(temp_ym42_1_mod_month)

xt_ocean = temp_ym42_1_mod_month.xt_ocean
xt_ocean_corrected = xt_ocean + 360
temp_ym42_1_mod_month = temp_ym42_1_mod_month.assign_coords(xt_ocean=xt_ocean_corrected)
st_ocean = temp_ym42_1_mod_month.st_ocean
st_ocean_corrected = -st_ocean
temp_ym42_1_mod_month = temp_ym42_1_mod_month.assign_coords(st_ocean=st_ocean_corrected)
print(temp_ym42_1_mod_month)

temp_ym42_1_mod = temp_ym42_1_mod_month.groupby('time.year').mean('time')
print(temp_ym42_1_mod)

temp_ym42_1_mod = temp_ym42_1_mod.sel(year=slice(2004+60*4,2017+60*4))
print(temp_ym42_1_mod)


ekey='025deg'
expt = exptdata.exptdict[ekey]['expt']
print(expt)

temp_ym42_025_mod_month = cc.querying.getvar(expt=expt,
                                     session=session,
                                     ncfile='ocean.nc',
                                     variable='temp').\
interp(yt_ocean=latL).\
sel(xt_ocean=slice(lonW,lonE)).\
sel(st_ocean=slice(0,2100))-273.15
print(temp_ym42_025_mod_month)

temp_ym42_025_mod_month.load()
print(temp_ym42_025_mod_month)

xt_ocean = temp_ym42_025_mod_month.xt_ocean
xt_ocean_corrected = xt_ocean + 360
temp_ym42_025_mod_month = temp_ym42_025_mod_month.assign_coords(xt_ocean=xt_ocean_corrected)
st_ocean = temp_ym42_025_mod_month.st_ocean
st_ocean_corrected = -st_ocean
temp_ym42_025_mod_month = temp_ym42_025_mod_month.assign_coords(st_ocean=st_ocean_corrected)
print(temp_ym42_025_mod_month)

temp_ym42_025_mod = temp_ym42_025_mod_month.groupby('time.year').mean('time')
print(temp_ym42_025_mod)

temp_ym42_025_mod = temp_ym42_025_mod.sel(year=slice(2004+60*4,2017+60*4))
print(temp_ym42_025_mod)


ekey='01deg'
expt = exptdata.exptdict[ekey]['expt']
print(expt)

temp_ym42_01_mod_month = cc.querying.getvar(expt=expt,
                                     session=session,
                                     ncfile='ocean.nc',
                                     variable='temp').\
sel(yt_ocean=latL, method='nearest').\
sel(xt_ocean=slice(lonW,lonE)).\
sel(st_ocean=slice(0,2100))-273.15
print(temp_ym42_01_mod_month)

temp_ym42_01_mod_month.load()
print(temp_ym42_01_mod_month)

xt_ocean = temp_ym42_01_mod_month.xt_ocean
xt_ocean_corrected = xt_ocean + 360
temp_ym42_01_mod_month = temp_ym42_01_mod_month.assign_coords(xt_ocean=xt_ocean_corrected)
st_ocean = temp_ym42_01_mod_month.st_ocean
st_ocean_corrected = -st_ocean
temp_ym42_01_mod_month = temp_ym42_01_mod_month.assign_coords(st_ocean=st_ocean_corrected)
print(temp_ym42_01_mod_month)

temp_ym42_01_mod = temp_ym42_01_mod_month.groupby('time.year').mean('time')
print(temp_ym42_01_mod)

temp_ym42_01_mod = temp_ym42_01_mod.sel(year=slice(2004,2017))
print(temp_ym42_01_mod)


year = temp_ym42_1_mod.year
depth_1_mod = temp_ym42_1_mod.st_ocean
lon_1_mod = temp_ym42_1_mod.xt_ocean
temp_ym42_1_trans_mod = temp_ym42_1_mod
print(np.shape(temp_ym42_1_trans_mod))
temp_ym42_1_slope_mod = xr.DataArray(
    np.zeros([np.shape(depth_1_mod)[0], np.shape(lon_1_mod)[0]]), dims=(
    'depth_1', 'lon_1'), coords=[depth_1_mod, lon_1_mod], name='temp_ym42_1')
temp_ym42_1_p_value_mod = xr.DataArray(
    np.zeros([np.shape(depth_1_mod)[0], np.shape(lon_1_mod)[0]]), dims=(
    'depth_1', 'lon_1'), coords=[depth_1_mod, lon_1_mod], name='temp_ym42_1')
temp_ym42_1_std_err_mod = xr.DataArray(
    np.zeros([np.shape(depth_1_mod)[0], np.shape(lon_1_mod)[0]]), dims=(
    'depth_1', 'lon_1'), coords=[depth_1_mod, lon_1_mod], name='temp_ym42_1')
for iid, i in enumerate(depth_1_mod):
    for jid, j in enumerate(lon_1_mod):
        temp_ym42_1_slope_mod[iid, jid], _, _, temp_ym42_1_p_value_mod[iid, jid], temp_ym42_1_std_err_mod[iid, jid] = \
        stats.linregress(year, temp_ym42_1_trans_mod[:, iid, jid])
    print('depth_1_mod ' + str(np.array(i)))
print(temp_ym42_1_slope_mod)
year = temp_ym42_025_mod.year
depth_025_mod = temp_ym42_025_mod.st_ocean
lon_025_mod = temp_ym42_025_mod.xt_ocean
temp_ym42_025_trans_mod = temp_ym42_025_mod
print(np.shape(temp_ym42_025_trans_mod))
temp_ym42_025_slope_mod = xr.DataArray(
    np.zeros([np.shape(depth_025_mod)[0], np.shape(lon_025_mod)[0]]), dims=(
    'depth_025', 'lon_025'), coords=[depth_025_mod, lon_025_mod], name='temp_ym42_025')
temp_ym42_025_p_value_mod = xr.DataArray(
    np.zeros([np.shape(depth_025_mod)[0], np.shape(lon_025_mod)[0]]), dims=(
    'depth_025', 'lon_025'), coords=[depth_025_mod, lon_025_mod], name='temp_ym42_025')
temp_ym42_025_std_err_mod = xr.DataArray(
    np.zeros([np.shape(depth_025_mod)[0], np.shape(lon_025_mod)[0]]), dims=(
    'depth_025', 'lon_025'), coords=[depth_025_mod, lon_025_mod], name='temp_ym42_025')
for iid, i in enumerate(depth_025_mod):
    for jid, j in enumerate(lon_025_mod):
        temp_ym42_025_slope_mod[iid, jid], _, _, temp_ym42_025_p_value_mod[iid, jid], temp_ym42_025_std_err_mod[iid, jid] = \
        stats.linregress(year, temp_ym42_025_trans_mod[:, iid, jid])
    print('depth_025_mod ' + str(np.array(i)))
print(temp_ym42_025_slope_mod)
year = temp_ym42_01_mod.year
depth_01_mod = temp_ym42_01_mod.st_ocean
lon_01_mod = temp_ym42_01_mod.xt_ocean
temp_ym42_01_trans_mod = temp_ym42_01_mod
print(np.shape(temp_ym42_01_trans_mod))
temp_ym42_01_slope_mod = xr.DataArray(
    np.zeros([np.shape(depth_01_mod)[0], np.shape(lon_01_mod)[0]]), dims=(
    'depth_01', 'lon_01'), coords=[depth_01_mod, lon_01_mod], name='temp_ym42_01')
temp_ym42_01_p_value_mod = xr.DataArray(
    np.zeros([np.shape(depth_01_mod)[0], np.shape(lon_01_mod)[0]]), dims=(
    'depth_01', 'lon_01'), coords=[depth_01_mod, lon_01_mod], name='temp_ym42_01')
temp_ym42_01_std_err_mod = xr.DataArray(
    np.zeros([np.shape(depth_01_mod)[0], np.shape(lon_01_mod)[0]]), dims=(
    'depth_01', 'lon_01'), coords=[depth_01_mod, lon_01_mod], name='temp_ym42_01')
for iid, i in enumerate(depth_01_mod):
    for jid, j in enumerate(lon_01_mod):
        temp_ym42_01_slope_mod[iid, jid], _, _, temp_ym42_01_p_value_mod[iid, jid], temp_ym42_01_std_err_mod[iid, jid] = \
        stats.linregress(year, temp_ym42_01_trans_mod[:, iid, jid])
    print('depth_01_mod ' + str(np.array(i)))
print(temp_ym42_01_slope_mod)

1deg_jra55v13_iaf_spinup1_B1


/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/cosima_cookbook/querying.py:96: FutureWarning: In xarray version 0.14 the default behaviour of `open_mfdataset`
will change. To retain the existing behavior, pass
combine='nested'. To use future default behavior, pass
combine='by_coords'. See
http://xarray.pydata.org/en/stable/combining.html#combining-multi

  preprocess=lambda d: d[variable].to_dataset() if variable not in d.coords else d)
/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/xarray/backends/api.py:934: FutureWarning: The datasets supplied have global dimension coordinates. You may want
to use the new `combine_by_coords` function (or the
`combine='by_coords'` option to `open_mfdataset`) to order the datasets
before concatenation. Alternatively, to continue concatenating based
on the order the datasets are supplied in future, please use the new
`combine_nested` function (or the `combine='nested'` option t

<xarray.DataArray 'temp' (time: 300, st_ocean: 35, xt_ocean: 30)>
dask.array<sub, shape=(300, 35, 30), dtype=float32, chunksize=(1, 25, 30), chunktype=numpy.ndarray>
Coordinates:
  * st_ocean  (st_ocean) float64 1.152 3.649 6.565 ... 1.854e+03 2.046e+03
  * xt_ocean  (xt_ocean) float64 -214.5 -213.5 -212.5 ... -187.5 -186.5 -185.5
  * time      (time) object 1958-07-02 12:00:00 ... 2257-07-02 12:00:00
    yt_ocean  int64 -42
<xarray.DataArray 'temp' (time: 300, st_ocean: 35, xt_ocean: 30)>
array([[[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ...

/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)
/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/cosima_cookbook/querying.py:96: FutureWarning: In xarray version 0.14 the default behaviour of `open_mfdataset`
will change. To retain the existing behavior, pass
combine='nested'. To use future default behavior, pass
combine='by_coords'. See
http://xarray.pydata.org/en/stable/combining.html#combining-multi

  preprocess=lambda d: d[variable].to_dataset() if variable not in d.coords else d)
/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/xarray/backends/api.py:934: FutureWarning: The datasets supplied have global dimension coordinates. You may want
to use the new `combine_by_coords` function (or the
`combine='by_coords'` option to `open_mfdataset`) to order the datasets
befo

<xarray.DataArray 'temp' (time: 300, st_ocean: 35, xt_ocean: 120)>
dask.array<sub, shape=(300, 35, 120), dtype=float32, chunksize=(1, 10, 92), chunktype=numpy.ndarray>
Coordinates:
  * st_ocean  (st_ocean) float64 1.152 3.649 6.565 ... 1.854e+03 2.046e+03
  * xt_ocean  (xt_ocean) float64 -214.9 -214.6 -214.4 ... -185.6 -185.4 -185.1
  * time      (time) object 1958-06-30 12:00:00 ... 2257-06-30 12:00:00
    yt_ocean  int64 -42
<xarray.DataArray 'temp' (time: 300, st_ocean: 35, xt_ocean: 120)>
array([[[13.00262571,         nan,         nan, ..., 13.16772088,
         13.45407194, 13.81226093],
        [12.99652567,         nan,         nan, ..., 13.15586231,
         13.44637072, 13.80760936],
        [12.98901375,         nan,         nan, ..., 13.13632982,
         13.43193652, 13.79625198],
        ...,
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
               

/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)
/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/cosima_cookbook/querying.py:96: FutureWarning: In xarray version 0.14 the default behaviour of `open_mfdataset`
will change. To retain the existing behavior, pass
combine='nested'. To use future default behavior, pass
combine='by_coords'. See
http://xarray.pydata.org/en/stable/combining.html#combining-multi

  preprocess=lambda d: d[variable].to_dataset() if variable not in d.coords else d)
/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/xarray/backends/api.py:934: FutureWarning: The datasets supplied have global dimension coordinates. You may want
to use the new `combine_by_coords` function (or the
`combine='by_coords'` option to `open_mfdataset`) to order the datasets
befo

<xarray.DataArray 'temp' (time: 396, st_ocean: 55, xt_ocean: 300)>
dask.array<sub, shape=(396, 55, 300), dtype=float32, chunksize=(1, 7, 150), chunktype=numpy.ndarray>
Coordinates:
    yt_ocean  float64 -41.99
  * st_ocean  (st_ocean) float64 0.5413 1.681 2.94 ... 1.858e+03 2.007e+03
  * xt_ocean  (xt_ocean) float64 -214.9 -214.8 -214.7 ... -185.2 -185.1 -185.0
  * time      (time) object 1985-01-14 12:00:00 ... 2017-12-14 12:00:00
<xarray.DataArray 'temp' (time: 396, st_ocean: 55, xt_ocean: 300)>
array([[[14.098328 , 14.1996765,        nan, ..., 17.375946 ,
         17.697144 , 17.855927 ],
        [14.090515 , 14.190582 ,        nan, ..., 17.366333 ,
         17.690308 , 17.847351 ],
        [14.081116 , 14.178864 ,        nan, ..., 17.356964 ,
         17.682983 , 17.83847  ],
        ...,
        [       nan,        nan,        nan, ...,        nan,
                nan,  2.4491882],
        [       nan,        nan,        nan, ...,        nan,
                nan,  2.1709595],
    

/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)


depth_1_mod -1.15174989182586
depth_1_mod -3.6486741197120196
depth_1_mod -6.564918471676153
depth_1_mod -9.970869194872916
depth_1_mod -13.948708373749158
depth_1_mod -18.594379460593196
depth_1_mod -24.019873617545198
depth_1_mod -30.355885242559452
depth_1_mod -37.754889932903495
depth_1_mod -46.39470194845016
depth_1_mod -56.48256969937929
depth_1_mod -68.25986482598941
depth_1_mod -82.00740970018322
depth_1_mod -98.05146439730592
depth_1_mod -116.77034928747688
depth_1_mod -138.60160118643546
depth_1_mod -164.04943216631892
depth_1_mod -193.69205664349184
depth_1_mod -228.1881444592732
depth_1_mod -268.2812141854562
depth_1_mod -314.800181990729
depth_1_mod -368.65354740782504
depth_1_mod -430.8139438640983
depth_1_mod -502.28929308570173
depth_1_mod -584.0771003035616
depth_1_mod -677.1002526914099
depth_1_mod -782.1267221440287
depth_1_mod -899.6817765561261
depth_1_mod -1029.9681126743835
depth_1_mod -1172.813371535859
depth_1_mod -1327.6620295174746
depth_1_mod -1493.618454761

In [13]:
temp_ym42_1_mod_xr = xr.DataArray(temp_ym42_1_mod.transpose('st_ocean', 'xt_ocean', 'year'), name='temp_ym42_1_mod',
                        coords=[depth_1_mod, lon_1_mod, year], 
                        dims=['st_ocean', 'xt_ocean', 'year'])
print(temp_ym42_1_mod_xr)
temp_ym42_1_mod_xr.to_netcdf(output_path + 'temp_ym42_1_mod.nc')

temp_ym42_1_slope_mod_xr = xr.DataArray(temp_ym42_1_slope_mod, name='temp_ym42_1_slope_mod',
                        coords=[depth_1_mod, lon_1_mod], 
                        dims=['st_ocean', 'xt_ocean'])
print(temp_ym42_1_slope_mod_xr)
temp_ym42_1_slope_mod_xr.to_netcdf(output_path + 'temp_ym42_1_slope_mod.nc')

temp_ym42_1_p_value_mod_xr = xr.DataArray(temp_ym42_1_p_value_mod, name='temp_ym42_1_p_value_mod',
                        coords=[depth_1_mod, lon_1_mod], 
                        dims=['st_ocean', 'xt_ocean'])
print(temp_ym42_1_p_value_mod_xr)
temp_ym42_1_p_value_mod_xr.to_netcdf(output_path + 'temp_ym42_1_p_value_mod.nc')

temp_ym42_1_std_err_mod_xr = xr.DataArray(temp_ym42_1_std_err_mod, name='temp_ym42_1_std_err_mod',
                        coords=[depth_1_mod, lon_1_mod], 
                        dims=['st_ocean', 'xt_ocean'])
print(temp_ym42_1_std_err_mod_xr)
temp_ym42_1_std_err_mod_xr.to_netcdf(output_path + 'temp_ym42_1_std_err_mod.nc')



temp_ym42_025_mod_xr = xr.DataArray(temp_ym42_025_mod.transpose('st_ocean', 'xt_ocean', 'year'), name='temp_ym42_025_mod',
                        coords=[depth_025_mod, lon_025_mod, year], 
                        dims=['st_ocean', 'xt_ocean', 'year'])
print(temp_ym42_025_mod_xr)
temp_ym42_025_mod_xr.to_netcdf(output_path + 'temp_ym42_025_mod.nc')

temp_ym42_025_slope_mod_xr = xr.DataArray(temp_ym42_025_slope_mod, name='temp_ym42_025_slope_mod',
                        coords=[depth_025_mod, lon_025_mod], 
                        dims=['st_ocean', 'xt_ocean'])
print(temp_ym42_025_slope_mod_xr)
temp_ym42_025_slope_mod_xr.to_netcdf(output_path + 'temp_ym42_025_slope_mod.nc')

temp_ym42_025_p_value_mod_xr = xr.DataArray(temp_ym42_025_p_value_mod, name='temp_ym42_025_p_value_mod',
                        coords=[depth_025_mod, lon_025_mod], 
                        dims=['st_ocean', 'xt_ocean'])
print(temp_ym42_025_p_value_mod_xr)
temp_ym42_025_p_value_mod_xr.to_netcdf(output_path + 'temp_ym42_025_p_value_mod.nc')

temp_ym42_025_std_err_mod_xr = xr.DataArray(temp_ym42_025_std_err_mod, name='temp_ym42_025_std_err_mod',
                        coords=[depth_025_mod, lon_025_mod], 
                        dims=['st_ocean', 'xt_ocean'])
print(temp_ym42_025_std_err_mod_xr)
temp_ym42_025_std_err_mod_xr.to_netcdf(output_path + 'temp_ym42_025_std_err_mod.nc')



temp_ym42_01_mod_xr = xr.DataArray(temp_ym42_01_mod.transpose('st_ocean', 'xt_ocean', 'year'), name='temp_ym42_01_mod',
                        coords=[depth_01_mod, lon_01_mod, year], 
                        dims=['st_ocean', 'xt_ocean', 'year'])
print(temp_ym42_01_mod_xr)
temp_ym42_01_mod_xr.to_netcdf(output_path + 'temp_ym42_01_mod.nc')

temp_ym42_01_slope_mod_xr = xr.DataArray(temp_ym42_01_slope_mod, name='temp_ym42_01_slope_mod',
                        coords=[depth_01_mod, lon_01_mod], 
                        dims=['st_ocean', 'xt_ocean'])
print(temp_ym42_01_slope_mod_xr)
temp_ym42_01_slope_mod_xr.to_netcdf(output_path + 'temp_ym42_01_slope_mod.nc')

temp_ym42_01_p_value_mod_xr = xr.DataArray(temp_ym42_01_p_value_mod, name='temp_ym42_01_p_value_mod',
                        coords=[depth_01_mod, lon_01_mod], 
                        dims=['st_ocean', 'xt_ocean'])
print(temp_ym42_01_p_value_mod_xr)
temp_ym42_01_p_value_mod_xr.to_netcdf(output_path + 'temp_ym42_01_p_value_mod.nc')

temp_ym42_01_std_err_mod_xr = xr.DataArray(temp_ym42_01_std_err_mod, name='temp_ym42_01_std_err_mod',
                        coords=[depth_01_mod, lon_01_mod], 
                        dims=['st_ocean', 'xt_ocean'])
print(temp_ym42_01_std_err_mod_xr)
temp_ym42_01_std_err_mod_xr.to_netcdf(output_path + 'temp_ym42_01_std_err_mod.nc')

<xarray.DataArray 'temp_ym42_1_mod' (st_ocean: 35, xt_ocean: 30, year: 14)>
array([[[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       ...,

       [[nan, nan, nan, ..., nan, nan, nan],
        [na

<xarray.DataArray 'temp_ym42_01_slope_mod' (st_ocean: 55, xt_ocean: 300)>
array([[0.04529041, 0.04421716,        nan, ..., 0.03652746, 0.03151261,
        0.02347541],
       [0.04518189, 0.04416156,        nan, ..., 0.03652116, 0.03151976,
        0.02345203],
       [0.04513451, 0.04418134,        nan, ..., 0.03648363, 0.03148231,
        0.02338148],
       ...,
       [       nan,        nan,        nan, ...,        nan,        nan,
        0.00165847],
       [       nan,        nan,        nan, ...,        nan,        nan,
        0.0009662 ],
       [       nan,        nan,        nan, ...,        nan,        nan,
        0.00045946]])
Coordinates:
  * st_ocean  (st_ocean) float64 -0.5413 -1.681 -2.94 ... -1.858e+03 -2.007e+03
  * xt_ocean  (xt_ocean) float64 145.1 145.2 145.3 145.4 ... 174.8 174.9 175.0
<xarray.DataArray 'temp_ym42_01_p_value_mod' (st_ocean: 55, xt_ocean: 300)>
array([[0.02845935, 0.0287986 ,        nan, ..., 0.0608703 , 0.10285899,
        0.18936995],
       

In [16]:
lonW = 145
lonE = 175

latL = -34

input_path = '/g/data/e14/erd561/ARGO/'
temp_LTM = xr.open_dataset(input_path + 'RG_ArgoClim_Temperature_2017.nc', decode_times=False).\
                ARGO_TEMPERATURE_MEAN.\
                sel(LONGITUDE=slice(lonW, lonE)).\
                interp(LATITUDE=latL)
print(temp_LTM)


temp_months = xr.open_dataset(input_path + 'RG_ArgoClim_Temperature_2017.nc', decode_times=False).\
                ARGO_TEMPERATURE_ANOMALY.\
                sel(LONGITUDE=slice(lonW, lonE)).\
                interp(LATITUDE=latL)
# print(temp_months)

TIME = [pd.date_range('15/1/2004',periods=156, freq=pd.DateOffset(months=1))]
# print(TIME)

temp_months = temp_months.assign_coords(TIME=np.squeeze(TIME))
# print(temp_months)

temp = temp_months.groupby('TIME.year').mean('TIME', skipna=True)
# print(temp)

temp_2004_2016 = temp.sel(year=slice(1993,2017))
print(temp_2004_2016)



temp_2017 = xr.open_mfdataset(sorted(gb(input_path + 'RG_ArgoClim_2017*.nc')), decode_times=False).\
                ARGO_TEMPERATURE_ANOMALY.\
                sel(LONGITUDE=slice(lonW, lonE)).\
                interp(LATITUDE=latL).\
                assign_coords(TIME=np.squeeze([pd.date_range('15/1/2017',periods=12, freq=pd.DateOffset(months=1))])).\
                groupby('TIME.year').mean('TIME', skipna=True)
temp_2017.load()
print(temp_2017)


temp = xr.concat([temp_2004_2016, temp_2017], dim='year') + temp_LTM
# print(temp)

temp = temp.assign_coords(PRESSURE=gsw.z_from_p(temp.PRESSURE, np.mean(temp.LATITUDE)))
print(temp)

year = temp.year
depth = temp.PRESSURE
lon = temp.LONGITUDE

# temp_trans = np.transpose(temp, (2,1,0))
temp_trans = temp
print(np.shape(temp_trans))
temp_slope = xr.DataArray(
    np.zeros([np.shape(depth)[0], np.shape(lon)[0]]), dims=(
    'depth', 'lon'), coords=[depth, lon], name='temp')
temp_p_value = xr.DataArray(
    np.zeros([np.shape(depth)[0], np.shape(lon)[0]]), dims=(
    'depth', 'lon'), coords=[depth, lon], name='temp')
temp_std_err = xr.DataArray(
    np.zeros([np.shape(depth)[0], np.shape(lon)[0]]), dims=(
    'depth', 'lon'), coords=[depth, lon], name='temp')
for iid, i in enumerate(depth):
    for jid, j in enumerate(lon):
        temp_slope[iid, jid], _, _, temp_p_value[iid, jid], temp_std_err[iid, jid] = \
        stats.linregress(year, temp_trans[:, iid, jid])
    print('depth ' + str(np.array(i)))
print(temp_slope)

temp_ym34_argo = temp
temp_ym34_slope_argo = temp_slope
temp_ym34_p_value_argo = temp_p_value
temp_ym34_std_err_argo = temp_std_err

temp_ym34_argo.to_netcdf(output_path + 'temp_ym34_argo.nc')
temp_ym34_slope_argo.to_netcdf(output_path + 'temp_ym34_slope_argo.nc')
temp_ym34_p_value_argo.to_netcdf(output_path + 'temp_ym34_p_value_argo.nc')
temp_ym34_std_err_argo.to_netcdf(output_path + 'temp_ym34_std_err_argo.nc')

<xarray.DataArray 'ARGO_TEMPERATURE_MEAN' (PRESSURE: 58, LONGITUDE: 30)>
array([[        nan,         nan,         nan, ..., 19.01399994,
        19.14900017, 19.14849949],
       [        nan,         nan,         nan, ..., 18.97399998,
        19.11450005, 19.11800003],
       [        nan,         nan,         nan, ..., 18.86800003,
        19.02299976, 19.03800011],
       ...,
       [        nan,         nan,         nan, ...,  2.68549991,
         2.67850006,  2.67150009],
       [        nan,         nan,         nan, ...,  2.50849998,
         2.4964999 ,  2.49249995],
       [        nan,         nan,         nan, ...,  2.398     ,
         2.38849998,  2.3865    ]])
Coordinates:
  * LONGITUDE  (LONGITUDE) float32 145.5 146.5 147.5 148.5 ... 172.5 173.5 174.5
  * PRESSURE   (PRESSURE) float32 2.5 10.0 20.0 30.0 ... 1800.0 1900.0 1975.0
    LATITUDE   int64 -34
Attributes:
    units:      degree celcius (ITS-90)
    long_name:  ARGO TEMPERATURE MEAN Jan 2004 - Dec 2016 (13.0 y

/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)
/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/ipykernel_launcher.py:34: FutureWarning: In xarray version 0.14 the default behaviour of `open_mfdataset`
will change. To retain the existing behavior, pass
combine='nested'. To use future default behavior, pass
combine='by_coords'. See
http://xarray.pydata.org/en/stable/combining.html#combining-multi

/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/xarray/backends/api.py:934: FutureWarning: The datasets supplied have global dimension coordinates. You may want
to use the new `combine_by_coords` function (or the
`combine='by_coords'` option to `open_mfdataset`) to order the datasets
before concatenation. Alternatively, to continue concatenating based
on the order the datasets

<xarray.DataArray 'ARGO_TEMPERATURE_ANOMALY' (year: 13, PRESSURE: 58, LONGITUDE: 30)>
array([[[            nan,             nan,             nan, ...,
         -1.37291652e-01, -1.64416673e-01, -1.63749993e-01],
        [            nan,             nan,             nan, ...,
         -1.30874985e-01, -1.55041688e-01, -1.55499990e-01],
        [            nan,             nan,             nan, ...,
         -1.16499991e-01, -1.34541655e-01, -1.37708326e-01],
        ...,
        [            nan,             nan,             nan, ...,
          7.91666515e-04,  4.79166673e-03,  6.95833356e-03],
        [            nan,             nan,             nan, ...,
          5.41666610e-04,  4.87500007e-03,  6.16666679e-03],
        [            nan,             nan,             nan, ...,
         -1.62500005e-03, -7.08333304e-04, -1.45833329e-03]],

       [[            nan,             nan,             nan, ...,
          6.09583631e-02, -2.17500019e-02, -3.12500037e-02],
        [        

depth -9.928426676694892
depth -19.856371271868333
depth -29.783833934280278
depth -39.71081481264839
depth -49.63731405564806
depth -59.56333181191236
depth -69.48886823003207
depth -79.4139234585557
depth -89.33849764598939
depth -99.26259094079703
depth -109.18620349140006
depth -119.10933544617778
depth -129.03198695346694
depth -138.95415816156205
depth -148.8758492187153
depth -158.7970602731364
depth -168.71779147299284
depth -181.11803090298696
depth -198.4771074262124
depth -218.31425483669733
depth -238.14948638133842
depth -257.9828032429312
depth -277.81420660359186
depth -297.64369764475816
depth -317.47127754718775
depth -337.29694749095927
depth -357.1207086554713
depth -376.9425622194421
depth -396.76250936091037
depth -416.58055125723376
depth -436.3966890850891
depth -458.6875696488121
depth -495.8336899144017
depth -545.3514644324973
depth -594.8573862194971
depth -644.3514735730843
depth -693.8337447642567
depth -743.3042180373038
depth -792.7629116097823
depth -842

In [18]:
lonW = 145
lonE = 175

latL = -42

input_path = '/g/data/e14/erd561/ARGO/'
temp_LTM = xr.open_dataset(input_path + 'RG_ArgoClim_Temperature_2017.nc', decode_times=False).\
                ARGO_TEMPERATURE_MEAN.\
                sel(LONGITUDE=slice(lonW, lonE)).\
                interp(LATITUDE=latL)
print(temp_LTM)


temp_months = xr.open_dataset(input_path + 'RG_ArgoClim_Temperature_2017.nc', decode_times=False).\
                ARGO_TEMPERATURE_ANOMALY.\
                sel(LONGITUDE=slice(lonW, lonE)).\
                interp(LATITUDE=latL)
# print(temp_months)

TIME = [pd.date_range('15/1/2004',periods=156, freq=pd.DateOffset(months=1))]
# print(TIME)

temp_months = temp_months.assign_coords(TIME=np.squeeze(TIME))
# print(temp_months)

temp = temp_months.groupby('TIME.year').mean('TIME', skipna=True)
# print(temp)

temp_2004_2016 = temp.sel(year=slice(1993,2017))
print(temp_2004_2016)



temp_2017 = xr.open_mfdataset(sorted(gb(input_path + 'RG_ArgoClim_2017*.nc')), decode_times=False).\
                ARGO_TEMPERATURE_ANOMALY.\
                sel(LONGITUDE=slice(lonW, lonE)).\
                interp(LATITUDE=latL).\
                assign_coords(TIME=np.squeeze([pd.date_range('15/1/2017',periods=12, freq=pd.DateOffset(months=1))])).\
                groupby('TIME.year').mean('TIME', skipna=True)
temp_2017.load()
print(temp_2017)


temp = xr.concat([temp_2004_2016, temp_2017], dim='year') + temp_LTM
# print(temp)

temp = temp.assign_coords(PRESSURE=gsw.z_from_p(temp.PRESSURE, np.mean(temp.LATITUDE)))
print(temp)

year = temp.year
depth = temp.PRESSURE
lon = temp.LONGITUDE

# temp_trans = np.transpose(temp, (2,1,0))
temp_trans = temp
print(np.shape(temp_trans))
temp_slope = xr.DataArray(
    np.zeros([np.shape(depth)[0], np.shape(lon)[0]]), dims=(
    'depth', 'lon'), coords=[depth, lon], name='temp')
temp_p_value = xr.DataArray(
    np.zeros([np.shape(depth)[0], np.shape(lon)[0]]), dims=(
    'depth', 'lon'), coords=[depth, lon], name='temp')
temp_std_err = xr.DataArray(
    np.zeros([np.shape(depth)[0], np.shape(lon)[0]]), dims=(
    'depth', 'lon'), coords=[depth, lon], name='temp')
for iid, i in enumerate(depth):
    for jid, j in enumerate(lon):
        temp_slope[iid, jid], _, _, temp_p_value[iid, jid], temp_std_err[iid, jid] = \
        stats.linregress(year, temp_trans[:, iid, jid])
    print('depth ' + str(np.array(i)))
print(temp_slope)

temp_ym42_argo = temp
temp_ym42_slope_argo = temp_slope
temp_ym42_p_value_argo = temp_p_value
temp_ym42_std_err_argo = temp_std_err

temp_ym42_argo.to_netcdf(output_path + 'temp_ym42_argo.nc')
temp_ym42_slope_argo.to_netcdf(output_path + 'temp_ym42_slope_argo.nc')
temp_ym42_p_value_argo.to_netcdf(output_path + 'temp_ym42_p_value_argo.nc')
temp_ym42_std_err_argo.to_netcdf(output_path + 'temp_ym42_std_err_argo.nc')

<xarray.DataArray 'ARGO_TEMPERATURE_MEAN' (PRESSURE: 58, LONGITUDE: 30)>
array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]])
Coordinates:
  * LONGITUDE  (LONGITUDE) float32 145.5 146.5 147.5 148.5 ... 172.5 173.5 174.5
  * PRESSURE   (PRESSURE) float32 2.5 10.0 20.0 30.0 ... 1800.0 1900.0 1975.0
    LATITUDE   int64 -42
Attributes:
    units:      degree celcius (ITS-90)
    long_name:  ARGO TEMPERATURE MEAN Jan 2004 - Dec 2016 (13.0 year) RG CLIM...


/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)
/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/ipykernel_launcher.py:34: FutureWarning: In xarray version 0.14 the default behaviour of `open_mfdataset`
will change. To retain the existing behavior, pass
combine='nested'. To use future default behavior, pass
combine='by_coords'. See
http://xarray.pydata.org/en/stable/combining.html#combining-multi

/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/xarray/backends/api.py:934: FutureWarning: The datasets supplied have global dimension coordinates. You may want
to use the new `combine_by_coords` function (or the
`combine='by_coords'` option to `open_mfdataset`) to order the datasets
before concatenation. Alternatively, to continue concatenating based
on the order the datasets

<xarray.DataArray 'ARGO_TEMPERATURE_ANOMALY' (year: 13, PRESSURE: 58, LONGITUDE: 30)>
array([[[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       ...,

       [[nan, nan, nan, ..., nan, nan, nan],
 

TypeError: 'NoneType' object is not callable